In [ ]:
import pymc3 as pm
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

# データの特徴をつかむ

例：ベルヌーイ分布

表の確率0.3のコインを100回振ってサンプルを得る

In [ ]:
n_experiments = 100
theta_real = 0.3
data = stats.bernoulli.rvs(p=theta_real, size=n_experiments)
data

# モデルを決める

# 事後分布を求める

https://docs.pymc.io/notebooks/getting_started.html

In [ ]:
#概要
with pm.Model() as model:
    theta = pm.Beta('theta', alpha=1, beta=1) #事前分布
    y = pm.Bernoulli('y', p=theta, observed=data) #尤度
    
    start = pm.find_MAP() #初期値
    step = pm.Metropolis() #サンプリング法
    trace = pm.sample(1000)

In [ ]:
#モデルの形状
pm.model_to_graphviz(model)

## 初期値

デフォルトではBroyden–Fletcher–Goldfarb–Shanno (BFGS)によるMAP推定値

In [ ]:
#BFGS
map_estimate = pm.find_MAP(model=model)
map_estimate

In [ ]:
#powell's method
map_estimate = pm.find_MAP(model=model, method='powell')
map_estimate

## サンプリング法
デフォルトではNUTS（推奨？），メトロポリス法の指定可

In [ ]:
with pm.Model() as model:
    theta = pm.Beta('theta', alpha=1, beta=1) #事前分布
    y = pm.Bernoulli('y', p=theta, observed=data) #尤度
    
    trace = pm.sample(1000)

In [ ]:
trace

In [ ]:
#サンプリング結果
pm.traceplot(trace);

## バーンイン
サンプリングのはじめの方は不安定なので，サンプルから除く

In [ ]:
burnin = 100
chain = trace[burnin:]

pm.traceplot(chain);

## multi_trace
サンプル数を増やす

In [ ]:
with pm.Model() as model2:
    theta = pm.Beta('theta', alpha=1, beta=1)
    y = pm.Bernoulli('y', p=theta, observed=data)

    multi_trace = pm.sample(1000, njobs=4)
    
burnin = 100
multi_chain = multi_trace[burnin:]
pm.traceplot(multi_chain);

## 収束性のチェック

In [ ]:
#Gelman-Rubinテスト，1.1未満なら良い
pm.gelman_rubin(multi_chain) 

In [ ]:
pm.forestplot(multi_chain, varnames={'theta'});

plt.figure()

In [ ]:
#pm.summary(multi_chain)

In [ ]:
#自己相関，ないのが望ましい
pm.autocorrplot(multi_chain)

plt.figure()

事後分布の要約

In [ ]:
pm.plot_posterior(multi_chain, kde_plot=True)

plt.figure()

# 予測分布を求める

In [ ]:
ppc = pm.sample_posterior_predictive(multi_trace, samples=100, model=model)
ppc

In [ ]:
plt.xlim(0.0,1,0)

plt.hist([y.mean() for y in ppc['y']], bins=6);